In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch', 'port': 9200}])

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [109]:
import time
def timing(f):
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        print('⏰ {:s} function took {:.3f} ms'.format(f.__name__, (time2-time1)*1000.0))
        return ret
    return wrap

In [3]:
es.indices.get_alias("*")

{u'.apm-agent-configuration': {u'aliases': {}},
 u'.kibana_1': {u'aliases': {u'.kibana': {}}},
 u'.kibana_task_manager_1': {u'aliases': {u'.kibana_task_manager': {}}},
 u'montevideo': {u'aliases': {}}}

In [4]:
from elasticsearch.client import IndicesClient
ic = IndicesClient(es)

In [5]:
texts = [
   'Siento mucha tristeza al ver esto en Montevideo. Una persona compartió esta imagen que ocurrió hace minutos en la esquina de Salto y (Bernabé) Rivera. La cantidad de personas en situación de calle creció en la capital. Hay planes para mejorar esta realidad, pero duele verla.',
    'Mucha basura suelta por Varela frente al Policial',
    'En Montevideo podés ir a Porongos esquina Blandengues Upside-down face',
    '18 de Julio esquina Aquiles Lanza, Montevideo, Uruguay',
    'Roban estación de servicio  esquina Montevideo shopping. Hace una semana robaron una joyería a 50 metros de esa estación de servicio. Lugar de mucho tránsito de personas y vehículos. Lleno de camaras de video vigilancia y PADO Man police officerDown pointing backhand index',
    'En esta foto estas en que lugar de Europa, amorosa? Ésta estuvo en el cerro de montevideo, calle Venezuela esquina Bogotá....Jaaaa....sos una genia! Smiling face with open mouth and smiling eyes',
    '👷🏻‍| #MontevideoMejora Round pushpin Av. Italia de Gallinal a Bolivia, luce más amplia y linda Bus Un carril más para el bus Mejor captación de aguas pluviales 🅿 Refugios Bicisenda 40 nuevas columnas de alumbrado y 80 luminarias LED Más info:',
    '@quejasyaEn la esquina de Montevideo Shopping hay una "montaña" de basura (Galarza y Tiburcio Gómez)',
    'Cheering megaphone Modificación de paradas para líneas 147 y 148 hacia afuera Calendar Desde el 5 de marzo Cross mark Se suprime la parada de Paysandú esquina Tristán Narvaja Hourglass Funciona provisoriamente en Tristán Narvaja esquina Cerro Largo',
    'Un hecho histórico en la Medicina Mundial. Ciudad de Montevideo. Esquina de Colonia y Arenal Grande. CASMU 1',
    'Cno. de las Tropas esquina Verdún',
    '¡Hola! Para este tipo de errores en el registro debes acudir a nuestra oficina, allí te darán la información y la ayuda necesaria para corregir este error y completar tu proceso. Recuerda que nuestra oficina está ubicada en Montevideo, José Ellauri 938, esquina Benito Lamas. 🚴🏽Sparkling heart',
    '🚧❌ | MEDIA CALZADA 📍 26 DE MARZO entre L.A. de Herrera y Marco Bruto 🕖 De 7 a 16 h  📍 JOSÉ ELLAURI esquina Leyenda Patria 🕖 De 7 a 17 h  📍 LAGUNILLAS esquina Joaquín Núñez 🕖 De 8 a 18 h  📍 8 DE OCTUBRE desde 18 de Julio hasta Colonia 🕖 De 8 a 18 h',
    'Speaker with three sound waves “Durazno y Convención” es una canción de 1984, que hoy es un tema que se puede escuchar mismo en la esquina sobre la que habla la letraRound pushpin  Ya que es un punto de Montevideo Sonoro Mobile phone Headphone',
    'En la ciudad de Montevideo, hace ya bastante tiempo, están dibujados los hongos de Super Mario Bros. En avenida Uruguay esquina Arenal Grande, está dibujado un hongo rojo. En avenida San Martín esquina Libres, está dibujado un hongo verde. Smiling face with heart-shaped eyes',
    'Alquiler anual. Centro de Montevideo 1 dormitorio $19.000 Héctor Gutiérrez Ruiz esquina San José Coordine visitas al 094283539 Llamadas o wsp',
    '¿Sos marica que usás vestido? Utilizando ésta frase tres efectivos de la Guardia Republicana apalearon a Joel, artista callejero que trabaja habitualmente en Millán y Garzón.'
]

In [6]:
def analyzedText(text):
    results = ic.analyze(index="montevideo", body=
    {
      "text": text, 
        "analyzer":"calle_analyzer"
    })
    return ' '.join([token['token'] for token in results['tokens']])

In [7]:
#@timing
def search(text):
    return es.search(index="montevideo", body=
        {
            "from" : 0, "size" : 500,
              "query": {
                "simple_query_string" : {
                    "query": text,
                    "analyzer": "calle_analyzer"
                }
              }
        })['hits']['hits']

In [8]:
#@timing
def boosting_search(text, size=50):
    return es.search(index="montevideo", body=
{
    "from" : 0, "size" : size,
    "query": {
        "boosting" : {
            "positive" : {
                "simple_query_string" : {
                    "query": text,
                    "analyzer": "calle_analyzer"
                }
            },
            "negative" : {
                "match" : {
                    "type" : "geonames_uy_montevideo limites_barrios"
                }
            },
            "negative_boost" : 0.5
        }
    }
})['hits']['hits']

In [9]:
#@timing
def boosting_match_search(text, size=150):
    return es.search(index="montevideo", body=
{
    "from" : 0, "size" : size,
    "query": {
        "boosting" : {
            "positive" : {
                "multi_match" : {
                    "query": text,
                    "analyzer": "calle_analyzer",
                    "fields": [ "nombre", "aliases" ],
                    "type": "best_fields",
                    
                }
            },
            "negative" : {
                "match" : {
                    "type" : "geonames_uy_montevideo limites_barrios v_mdg_espacios_libres"
                }
            },
            "negative_boost" : 0.5
        }
    }
})['hits']['hits']

In [96]:
#@timing
def boosting_match_bool_search(text, size=250):
    return es.search(index="montevideo", body=
{
    "from" : 0, "size" : size,
    "query": {
        "boosting" : {
            "positive" : {
                "bool" : {
                  "must" : {
                    "multi_match" : {
                        "query": text,
                        "analyzer": "calle_analyzer",
                        "fields": [ "nombre", "aliases" ],
                        "type": "best_fields"
                    
                    }
                  },
                  "must_not" : {
                    "multi_match" : {
                        "query": "la el la las los calle psje 1 mas",
                        "analyzer": "calle_analyzer",
                        "fields": [ "nombre", "aliases" ],
                        "type": "best_fields"
                    }
                  },
                "boost" : 2.0
                }
            },
            "negative" : {
                "match" : {
                    "type" : "geonames_uy_montevideo limites_barrios v_mdg_espacios_libres"
                }
            },
            "negative_boost" : 0.5
        }
    },
    "highlight": {
        "fields": {
            "nombre":{},
        }
    }
})['hits']['hits']

In [11]:
#@timing
def search_geo_vias(id,size=10):
    return es.search(index="montevideo", body=
    {
        "from" : 0, "size" : size,
        "query" : {
            "bool": {
                "must": {
                    "match" : {
                        "type" : "v_mdg_vias"
                    }
                },
                "filter": {
                    "geo_shape": {
                        "geometry": {
                            "indexed_shape": {
                                "index": "montevideo",
                                "id": id,
                                "path": "geometry"
                            }
                        }
                    }
                }
            }
        }
    })['hits']['hits']

In [12]:
def stripName(name):
    return name.replace(' ','-').strip().lower()

In [13]:
def getMatchName(result_obj_1,result_obj_2):
    return result_obj_1['s_name'] + '|' + result_obj_2['s_name']

In [14]:
def matchedName(match_dict,result_object):
    matched_key = next((key for key in match_dict.keys() if result_object['s_name'] in key), None)
    return matched_key!=None

In [15]:
def getMatchObj(res_obj,geo_obj=None):
    if geo_obj:
        match_score = res_obj['score']+geo_obj['score']
        match_objects = [res_obj,geo_obj]
        return {'score':match_score, 'objects':match_objects}
    else:
        return {'score':res_obj['score'], 'objects':[res_obj]} 

In [88]:
from difflib import SequenceMatcher
def commonSignificantSubstringExists(text1,text2,limit=5):
    match = SequenceMatcher(None, text1, text2).find_longest_match(0, len(text1), 0, len(text2))
    #print(text1[match.a: match.a + match.size])  # -> apple pie
    #print(text2[match.b: match.b + match.size])  # -> apple pie
    return match.size > limit

In [89]:
def sameHighlights(obj1,obj2):
    intersection = set(obj1['highlights']).intersection(obj2['highlights'])
    return len(list(intersection))==len(obj1['highlights'])

In [105]:
def tryToMatchLines(match_dict,line_results,all_results,favorMoreMatches=False):
    results = []
    if not favorMoreMatches:
        results = line_results.values()
    else:
        results = sorted(line_results.values(), key = lambda i: len(i['highlights']), reverse=True)
    for res_obj in results:
        if matchedName(match_dict, res_obj):
            #If there is alredy a match that involves this name continue
            continue
        for geo_result in search_geo_vias(res_obj['id']):
            #I only care for those objects intersecting current object AND where part of the original results.
            geo_obj = all_results.get(geo_result['_id'], None)
            if geo_obj and res_obj['s_name'] != geo_obj['s_name'] and not sameHighlights(res_obj,geo_obj) and not commonSignificantSubstringExists(res_obj['s_name'],geo_obj['s_name']):
                #I only need to care for different objects, matching names and not ids.
                #In case of streets more than one block can intersect with the next one, same name diff id.
                #I discard obj with same highlights or common significant substrings
                match_name = getMatchName(res_obj,geo_obj)
                match_obj = getMatchObj(res_obj,geo_obj)
                match_dict[match_name] = match_obj 

In [106]:
def tryToMatchPoints(match_dict,point_results,all_results):
    for key, res_obj in point_results.items():
        if matchedName(match_dict, res_obj):
            #If there is alredy a match that involves this name continue
            continue
        match_obj = getMatchObj(res_obj)
        match_dict[res_obj['s_name']] = match_obj

In [92]:
def getResultObject(result):
    result_geo_type = result['_source']['geometry']['type']
    result_id = result['_id']
    result_score = result['_score']
    result_geometry = result['_source']['geometry']
    result_name = 'NO_NAME'
    if result['_source'].get('nombre', None):
            result_name = result['_source']['nombre'].encode('ascii', 'ignore').decode('ascii')
    elif result['_source'].get('aliases',None):
            result_name = result['_source']['aliases'].encode('ascii', 'ignore').decode('ascii')
    result_striped_name = stripName(result_name)
    result_highlights = []
    highlights = result.get('highlight',None)
    if highlights:
        for a in highlights.values():
            for b in a:
                for c in b.split(' '):
                    if '<em>' in c:
                        result_highlights.append(c.replace('<em>','').replace('</em>',''))
    return {'id':result_id,'geo_type':result_geo_type,'name':result_name, 's_name':result_striped_name,'score':result_score,'geometry':result_geometry, 'highlights':result_highlights}

In [93]:
@timing
def complete_search(text,favorMoreMatches=False):  
    match_dict = {}
    
    line_results = {}
    point_results = {}
    polygon_results = {}
    all_results = {}

    results = boosting_match_bool_search(text)
    for result in results:
        result_object = getResultObject(result)
        if result_object['geo_type'] == 'LineString':
            line_results[result_object['id']] = result_object
        elif result_object['geo_type'] == 'Point':   
            point_results[result_object['id']] = result_object
        elif result_object['geo_type'] == 'Polygon':
            polygon_results[result_object['id']] = result_object
        else:
            print('Do i have other?')
        
    all_results.update(line_results)
    all_results.update(point_results)
    all_results.update(polygon_results)
    
    tryToMatchLines(match_dict,line_results,all_results,favorMoreMatches)
    tryToMatchPoints(match_dict,point_results,all_results)
    
    
    return results, sorted(match_dict.items(), key = lambda i: i[1]['score'], reverse=True)

In [110]:
for i,text in enumerate(texts):
    print(i,analyzedText(text))
    results, matches = complete_search(text)
    for match in matches[:4]:
        print ('✔️ MATCH: {}\t/\t{}'.format(match[1]['score'],match[0]))
    print('\n')

(0, u'siento mucha tristeza ver montevideo persona compartio imagen ocurrio hace minutos esquina salto bernabe rivera cantidad personas situacion crecio capital planes mejorar realidad duele verla')
⏰ complete_search function took 1786.242 ms
✔️ MATCH: 49.090118	/	bernabe-rivera|salto


(1, u'mucha basura suelta varela frente policial')
⏰ complete_search function took 585.572 ms
✔️ MATCH: 6.527852	/	hospital-policial-inspector-general-uruguay-genta
✔️ MATCH: 5.161053	/	monumento-a-jose-pedro-varela
✔️ MATCH: 5.161053	/	museo-pedagogico-jose-pedro-varela


(2, u'montevideo podes ir porongos esquina blandengues upside down face')
⏰ complete_search function took 141.887 ms
✔️ MATCH: 39.531998	/	porongos|blandengues
✔️ MATCH: 7.212454	/	esplendor-montevideo
✔️ MATCH: 7.212454	/	continental-(montevideo)
✔️ MATCH: 7.212454	/	after-montevideo


(3, u'18 julio esquina aquiles lanza montevideo uruguay')
⏰ complete_search function took 894.259 ms
✔️ MATCH: 42.711395	/	av-18-de-julio|dr-aquiles-r

In [111]:
for i,text in enumerate(texts):
    print(i,analyzedText(text))
    results, matches = complete_search(text,True)
    for match in matches[:4]:
        print ('✔️ MATCH: {}\t/\t{}'.format(match[1]['score'],match[0]))
    print('\n')

(0, u'siento mucha tristeza ver montevideo persona compartio imagen ocurrio hace minutos esquina salto bernabe rivera cantidad personas situacion crecio capital planes mejorar realidad duele verla')
⏰ complete_search function took 1335.103 ms
✔️ MATCH: 49.090118	/	bernabe-rivera|salto


(1, u'mucha basura suelta varela frente policial')
⏰ complete_search function took 509.728 ms
✔️ MATCH: 6.527852	/	hospital-policial-inspector-general-uruguay-genta
✔️ MATCH: 5.161053	/	monumento-a-jose-pedro-varela
✔️ MATCH: 5.161053	/	museo-pedagogico-jose-pedro-varela


(2, u'montevideo podes ir porongos esquina blandengues upside down face')
⏰ complete_search function took 145.835 ms
✔️ MATCH: 39.531998	/	porongos|blandengues
✔️ MATCH: 7.212454	/	esplendor-montevideo
✔️ MATCH: 7.212454	/	continental-(montevideo)
✔️ MATCH: 7.212454	/	after-montevideo


(3, u'18 julio esquina aquiles lanza montevideo uruguay')
⏰ complete_search function took 1059.607 ms
✔️ MATCH: 42.711395	/	av-18-de-julio|dr-aquiles-

In [82]:
text = texts[12]
print(analyzedText(text))
for res in boosting_match_bool_search(text,600):
    res = getResultObject(res)
    print(res['score'],res['name'], res['id'], res['highlights'])

🚧 ❌ media calzada 📍 26 marzo l.a herrera marco bruto 🕖 7 16 h 📍 jose ellauri esquina leyenda patria 🕖 7 17 h 📍 lagunillas esquina joaquin nunez 🕖 8 18 h 📍 8 octubre 18 julio colonia 🕖 8 18 h
⏰ boosting_match_bool_search function took 397.528 ms
(70.78259, u'H', u'v_mdg_vias-6874056', [u'H'])
(70.78259, u'H', u'v_mdg_vias-6874061', [u'H'])
(58.4268, u'CALLE H', u'v_mdg_vias-285723', [u'H'])
(58.4268, u'CALLE H', u'v_mdg_vias-5139611', [u'H'])
(58.4268, u'CALLE H', u'v_mdg_vias-5142204', [u'H'])
(55.538998, u'JOSE H FIGUEIRA', u'v_mdg_vias-296947', [u'JOSE', u'H'])
(55.538998, u'JOSE H FIGUEIRA', u'v_mdg_vias-296972', [u'JOSE', u'H'])
(55.538998, u'JOSE H FIGUEIRA', u'v_mdg_vias-297112', [u'JOSE', u'H'])
(55.538998, u'JOSE H FIGUEIRA', u'v_mdg_vias-297143', [u'JOSE', u'H'])
(55.538998, u'JOSE H FIGUEIRA', u'v_mdg_vias-7352929', [u'JOSE', u'H'])
(38.532166, u'CALLE 18', u'v_mdg_vias-280603', [u'18'])
(38.532166, u'CALLE 18', u'v_mdg_vias-4354250', [u'18'])
(38.532166, u'CALLE 18', u'v_mdg